<figure>
<img src="langchain_memory.png">
  <center><figcaption style="font-size:0.7em;">(https://python.langchain.com/v0.1/docs/modules/memory/)</figcaption></center>
</figure>

1. Import LLM
2. Bind LLM with Tool
3. Prompt template with history
4. Agent
5. Automated History Management With Sessions
6. Gradio UI
7. Persistent Memory

Chat Model

In [ ]:
!pip install langchain-openai

In [1]:
from langchain_openai import ChatOpenAI
import os

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                  api_key = os.environ.get("OPENAI_API_KEY"))

In [3]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca11243d-2cd1-486b-8efa-ee99153d6f02-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

Bind a tool with LLM

In [ ]:
!pip install wikipedia

In [4]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [5]:
wikipedia.run("President of India")

"Page: President of India\nSummary: The president of India (IAST: Bhārat kē Rāṣṭrapati) is the head of state of the Republic of India. The president is the nominal head of the executive, the first citizen of the country, as well as the supreme commander of the Indian Armed Forces. Droupadi Murmu is the 15th and current president, having taken office from 25 July 2022.\nThe office of president was created when India became a republic on 26 January 1950 when its constitution came into force. The president is indirectly elected by an electoral college comprising both houses of the Parliament of India and the legislative assemblies of each of India's states and territories, who themselves are all directly elected by the citizens.\nArticle 53 of the Constitution of India states that the president can exercise their powers directly or by subordinate authority, though all of the executive powers vested in the president are, in practice, exercised by the prime minister heading the Council of M

In [6]:
tools = [wikipedia]
llm_with_tools = llm.bind_tools(tools)

Create Prompt Template with History

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

Create Agent

In [8]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

In [9]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt | llm_with_tools | OpenAIToolsAgentOutputParser() )

In [10]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Chat with History

In [11]:
from langchain.memory import ChatMessageHistory
chat_history = ChatMessageHistory()

In [12]:
query = "Who won fifa 2018 world cup?"

response = agent_executor.invoke( {"input": query, "chat_history" : chat_history.messages})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'FIFA 2018 World Cup'}`


Page: 2018 FIFA World Cup
Summary: The 2018 FIFA World Cup was the 21st FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA. It took place in Russia from 14 June to 15 July 2018, after the country was awarded the hosting rights in late 2010. It was the eleventh time the championships had been held in Europe, the first time they were held in Eastern Europe, and the first time they were held across two continents (Europe and Asia). At an estimated cost of over $14.2 billion, it was the most expensive World Cup ever held until it was surpassed by the 2022 World Cup in Qatar.
The tournament phase involved 32 teams, of which 31 came through qualifying competitions, while the host nation Russia qualified automatically. Of the 32, 20 had also appeared in the 2014 event, while Iceland and Panama each made their debut at the World Cup. 64 matches 

In [13]:
chat_history.add_user_message(query)
chat_history.add_ai_message(response['output'])
print(chat_history)

Human: Who won fifa 2018 world cup?
AI: France won the FIFA 2018 World Cup by defeating Croatia in the final match with a score of 4-2. This victory marked France's second World Cup title.


In [14]:
query = "And which team croatia defeated in semis?"
response = agent_executor.invoke( {"input": query, "chat_history" : chat_history.messages})
print(response['output'])



> Entering new AgentExecutor chain...
Croatia defeated England in the semi-finals of the FIFA 2018 World Cup. They won the match 2-1 after extra time to secure their spot in the final against France.

> Finished chain.
Croatia defeated England in the semi-finals of the FIFA 2018 World Cup. They won the match 2-1 after extra time to secure their spot in the final against France.


Automated History Management With Sessions

In [15]:
store = {}
last_k_messages = 4

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    stored_messages = store[session_id].messages
    if len(stored_messages) >= 4:
        store[session_id].clear()

    for message in stored_messages[-last_k_messages:]:
        store[session_id].add_message(message)


    return store[session_id]

In [16]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [17]:
chain_with_message_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [18]:
response = chain_with_message_history.invoke(
                                        {"input": "hi"},
                                        {"configurable": {"session_id": "123"}},
                                        )

print(response)

Parent run ed8d7028-d966-48de-912e-c3f5583d938f not found for run 693c8802-dba6-4dd0-b18c-27c06da043ce. Treating as a root run.




> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.
{'input': 'hi', 'chat_history': [], 'output': 'Hello! How can I assist you today?'}


In [19]:
import logging
logging.getLogger().setLevel(logging.ERROR)

Showcase in UI

In [20]:
import gradio as gr
import uuid


with gr.Blocks() as demo:
    
    state = gr.State("")
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])


    def respond(message, chatbot_history, session_id):
        if not chatbot_history:
            session_id = uuid.uuid4().hex

        print("Session ID: ", session_id)

        response = chain_with_message_history.invoke(
                                        {"input": message},
                                        {"configurable": {"session_id": session_id}},
                                        )

        chatbot_history.append((message, response['output']))
        return "", chatbot_history, session_id

    msg.submit(respond, [msg, chatbot, state], [msg, chatbot, state])

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Session ID:  e87137dab46e401bb0dede32deae9462


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': '2018 FIFA World Cup'}`


Page: 2018 FIFA World Cup
Summary: The 2018 FIFA World Cup was the 21st FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA. It took place in Russia from 14 June to 15 July 2018, after the country was awarded the hosting rights in late 2010. It was the eleventh time the championships had been held in Europe, the first time they were held in Eastern Europe, and the first time they were held across two continents (Europe and Asia). At an estimated cost of over $14.2 billion, it was the most expensive World Cup ever held until it was surpassed by the 2022 World Cup in Qatar.
The tournament phase involved 32 teams, of which 31 came through qualifying competitions, while the host nation Russia qualified automatically. Of the 32, 20 had also appeared in the 2014 event, while Iceland and Panama each 

Persistent memory

In [23]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [24]:
chat_message_history = SQLChatMessageHistory(
    session_id="test_session", connection = "sqlite:///memory.db"
)

In [25]:
from sqlite3 import connect
con = connect('memory.db')

In [26]:
curr = con.execute("SELECT * from message_store;")
print(curr.fetchall())

[]


In [27]:
chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

In [28]:
curr = con.execute("SELECT * from message_store;")
print(curr.fetchall())

[(1, 'test_session', '{"type": "human", "data": {"content": "Hello", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'), (2, 'test_session', '{"type": "ai", "data": {"content": "Hi", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}')]


In [29]:
last_k_messages = 4


def get_session_history(session_id):
    chat_message_history = SQLChatMessageHistory(
    session_id=session_id, connection = "sqlite:///memory.db"
    )

    messages = chat_message_history.get_messages()
    chat_message_history.clear()
    
    for message in messages[-last_k_messages:]:
        chat_message_history.add_message(message)
    
    return chat_message_history


chain_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [30]:
query = "hi"
response = chain_with_history.invoke(
                                        {"input": query},
                                        {"configurable": {"session_id": "123"}},
                                        )



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


In [31]:
curr = con.execute("SELECT * from message_store;")
print(curr.fetchall())

[(1, 'test_session', '{"type": "human", "data": {"content": "Hello", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'), (2, 'test_session', '{"type": "ai", "data": {"content": "Hi", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}'), (3, '123', '{"type": "human", "data": {"content": "hi", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'), (4, '123', '{"type": "ai", "data": {"content": "Hello! How can I assist you today?", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}')]
